In [4]:
 ##Dataset final
import pandas as pd
import numpy as np

In [5]:
x_training_features=pd.read_csv("training_set_features_eda_notnulls.csv")
y_training_features=pd.read_csv("training_set_labels.csv")
#Los id no los necesitaremos al entrenar el modelo
x_training_features.drop(["Unnamed: 0.1","Unnamed: 0"], axis=1, inplace=True)

#Tomando en cuenta los features que están más correlacionados, se eliminarán behavioral_outside_home
# y doctor_recc_h1n1, tomando en cuenta tambien que tienen una menor correlación con las variables 
#targets frente a los features con los que se correlacionan.
x_training_features.drop(["behavioral_outside_home","doctor_recc_h1n1"], axis=1, inplace=True)

#Ahora solo necesitamos de las variables targets eliminar la columna que no necesitamos: los id
y_training_features.drop("respondent_id", axis=1, inplace=True)

In [6]:

from sklearn import preprocessing
l_encoder = preprocessing.LabelEncoder()

list_labels_encoders = ["age_group","education"]
list_dummies=["race","sex","marital_status","rent_or_own","employment_status","census_msa"]

# Se aplica label encoder a las variables ordinales
x_training_features["age_group"]=l_encoder.fit_transform(x_training_features["age_group"]) 
x_training_features["education"]=l_encoder.fit_transform(x_training_features["education"]) 

#se aplica one hot enocoder usando pandas: "Dummies"
x_training_features=pd.get_dummies(x_training_features, columns=list_dummies, prefix_sep='_')

In [7]:
#para evitar colinealidad debido a los dummies, se eliminará 1 columna por cada variable
#las columnas generadas por one hot encoder "dummies" que se eliminarán son:
list_dummies_drop=['race_Other or Multiple','sex_Male', \
            'marital_status_Not Married','rent_or_own_Own','employment_status_Unemployed', \
            'census_msa_Non-MSA']
x_training_features.drop(list_dummies_drop,axis=1, inplace=True)
x_training_features.columns

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_touch_face',
       'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months',
       'health_worker', 'health_insurance', 'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective', 'opinion_seas_risk',
       'opinion_seas_sick_from_vacc', 'age_group', 'education',
       'household_adults', 'household_children', 'race_Black', 'race_Hispanic',
       'race_White', 'sex_Female', 'marital_status_Married',
       'rent_or_own_Rent', 'employment_status_Employed',
       'employment_status_Not in Labor Force',
       'census_msa_MSA, Not Principle  City',
       'census_msa_MSA, Principle City'],
      dtype='object')

In [8]:
#Analizando presencia de valores atípicos
def find_outliers(df):
    outliers = {}
    for columna in df.columns:
        # Calcular el primer y tercer cuartil
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        # Calcular el rango intercuartílico (IQR)
        IQR = Q3 - Q1
        # Definir los límites para detectar outliers
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        # Encontrar los índices de los outliers
        outliers[columna] = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)].index.tolist()
        print(columna, "  , Cantidad de outliers: ", len(outliers[columna]))
    return outliers

    
outliers_per_feature = find_outliers(x_training_features)


h1n1_concern   , Cantidad de outliers:  0
h1n1_knowledge   , Cantidad de outliers:  0
behavioral_antiviral_meds   , Cantidad de outliers:  1301
behavioral_avoidance   , Cantidad de outliers:  0
behavioral_face_mask   , Cantidad de outliers:  1841
behavioral_wash_hands   , Cantidad de outliers:  4650
behavioral_large_gatherings   , Cantidad de outliers:  0
behavioral_touch_face   , Cantidad de outliers:  0
doctor_recc_seasonal   , Cantidad de outliers:  0
chronic_med_condition   , Cantidad de outliers:  0
child_under_6_months   , Cantidad de outliers:  2138
health_worker   , Cantidad de outliers:  2899
health_insurance   , Cantidad de outliers:  0
opinion_h1n1_vacc_effective   , Cantidad de outliers:  0
opinion_h1n1_risk   , Cantidad de outliers:  0
opinion_h1n1_sick_from_vacc   , Cantidad de outliers:  0
opinion_seas_vacc_effective   , Cantidad de outliers:  3427
opinion_seas_risk   , Cantidad de outliers:  0
opinion_seas_sick_from_vacc   , Cantidad de outliers:  6573
age_group   , Can

In [9]:
# Se realizarán dos dataframes, uno con los outliers y el otro sin ellos
#eliminando valores atipicos de la variable con más atipicos: opinion_seas_sick_from_vacc
#se espera posteriormente ver la posibilidad de mejora en el score de lo modelos al disminuir
#la cantidad de outliers
x_training_features_no_outliers = \
x_training_features.drop(outliers_per_feature["opinion_seas_sick_from_vacc"],axis=0)
find_outliers(x_training_features_no_outliers)
x_training_features_no_outliers.shape #20134 es la longitud

#los índices donde estaban los valores atípicos se utilizan para eliminarlos también 
#en el dataframe de la variable target
y_training_features_no_outliers = \
y_training_features.drop(outliers_per_feature["opinion_seas_sick_from_vacc"],axis=0)

#verificando que los dataframes tienen el mismo tamaño
x_training_features_no_outliers.shape[0] == y_training_features_no_outliers.shape[0]

h1n1_concern   , Cantidad de outliers:  0
h1n1_knowledge   , Cantidad de outliers:  0
behavioral_antiviral_meds   , Cantidad de outliers:  803
behavioral_avoidance   , Cantidad de outliers:  0
behavioral_face_mask   , Cantidad de outliers:  1161
behavioral_wash_hands   , Cantidad de outliers:  3825
behavioral_large_gatherings   , Cantidad de outliers:  0
behavioral_touch_face   , Cantidad de outliers:  0
doctor_recc_seasonal   , Cantidad de outliers:  0
chronic_med_condition   , Cantidad de outliers:  0
child_under_6_months   , Cantidad de outliers:  1487
health_worker   , Cantidad de outliers:  2199
health_insurance   , Cantidad de outliers:  0
opinion_h1n1_vacc_effective   , Cantidad de outliers:  0
opinion_h1n1_risk   , Cantidad de outliers:  0
opinion_h1n1_sick_from_vacc   , Cantidad de outliers:  3808
opinion_seas_vacc_effective   , Cantidad de outliers:  2521
opinion_seas_risk   , Cantidad de outliers:  0
opinion_seas_sick_from_vacc   , Cantidad de outliers:  0
age_group   , Cant

True

In [10]:
print("df con menos outliers: ",x_training_features_no_outliers.shape)
print("df con outliers: ",x_training_features.shape)

df con menos outliers:  (20134, 33)
df con outliers:  (26707, 33)


In [12]:
########################separando el train y test dataframe###############################
# ESTO SOLO SE HACE A MODO EJEMPLO, YA QUE LUEGO STRATIFIEDKFOLD SE APLICA DE MANERA CONJUNTA
#CON GRIDSERACH CV, ESTO SOLO PARA PRÓPOSITOS DE VERIFICAR SI ayuda a garantizar que cada fold 
#tenga una representación equitativa de todas las clases, evitando así la posibilidad de que un 
#fold tenga muy pocos ejemplos de una clase en particular.


#Ya que tenemos dos variables targets, haremos este proceso para cada una
y1=np.array(y_training_features['h1n1_vaccine'])
y2=np.array(y_training_features['seasonal_vaccine'])




from sklearn.model_selection import StratifiedKFold

############################## h1n1_vaccine ######################################
#splitting into 5 folds
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
train_list_index, test_list_index=[],[]
for i,(train_index, test_index) in enumerate(skf.split(x_training_features, y1)):
 print(f"Fold {i}:")
 print(f"  Train: index={train_index}")
 print(f"  Test:  index={test_index}")
 train_list_index.append(train_index) #saving the indexes od the 5 folds
 test_list_index.append(test_index)


Fold 0:
  Train: index=[    0     1     2 ... 26703 26704 26705]
  Test:  index=[    4     8    12 ... 26696 26701 26706]
Fold 1:
  Train: index=[    0     1     4 ... 26704 26705 26706]
  Test:  index=[    2     3     5 ... 26699 26700 26702]
Fold 2:
  Train: index=[    2     3     4 ... 26702 26705 26706]
  Test:  index=[    0     1     9 ... 26693 26703 26704]
Fold 3:
  Train: index=[    0     1     2 ... 26704 26705 26706]
  Test:  index=[    6    11    13 ... 26691 26695 26697]
Fold 4:
  Train: index=[    0     1     2 ... 26703 26704 26706]
  Test:  index=[    7    14    23 ... 26681 26692 26705]


In [17]:
############################## seasonal_vaccine ######################################
#splitting into 5 folds
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
train_list_index_2, test_list_index_2=[],[]
for i,(train_index, test_index) in enumerate(skf.split(x_training_features, y2)):
 print(f"Fold {i}:")
 print(f"  Train: index={train_index}")
 print(f"  Test:  index={test_index}")
 train_list_index_2.append(train_index) #saving the indexes od the 5 folds
 test_list_index_2.append(test_index)

Fold 0:
  Train: index=[    0     1     2 ... 26702 26705 26706]
  Test:  index=[    5     9    14 ... 26700 26703 26704]
Fold 1:
  Train: index=[    0     1     3 ... 26704 26705 26706]
  Test:  index=[    2    12    13 ... 26685 26694 26698]
Fold 2:
  Train: index=[    0     2     3 ... 26704 26705 26706]
  Test:  index=[    1     8    10 ... 26695 26699 26702]
Fold 3:
  Train: index=[    0     1     2 ... 26702 26703 26704]
  Test:  index=[    4    15    29 ... 26690 26705 26706]
Fold 4:
  Train: index=[    1     2     4 ... 26704 26705 26706]
  Test:  index=[    0     3     6 ... 26687 26696 26701]


In [18]:
#aplicando Standard Scaler to the 5 folds
from sklearn.preprocessing import StandardScaler
################################# h1n1_vaccine #######################################
x_train,x_test,y_train,y_test=[],[],[],[]
sc=StandardScaler()

for i in range(0, len(train_list_index)):
 x_train.append(sc.fit_transform(x_training_features.iloc[train_list_index[i]]))
 x_test.append(sc.transform(x_training_features.iloc[test_list_index[i]]))
 #target variable without standard scaler
 y_train.append(y_training_features.iloc[train_list_index[i]])
 y_test.append(y_training_features.iloc[test_list_index[i]])

In [19]:
#aplicando Standard Scaler to the 5 folds
################################# seasonal_vaccine #######################################
x_train_2,x_test_2,y_train_2,y_test_2=[],[],[],[]
sc=StandardScaler()

for i in range(0, len(train_list_index)):
 x_train_2.append(sc.fit_transform(x_training_features.iloc[train_list_index_2[i]]))
 x_test_2.append(sc.transform(x_training_features.iloc[test_list_index_2[i]]))
 #target variable without standard scaler
 y_train_2.append(y_training_features.iloc[train_list_index_2[i]])
 y_test_2.append(y_training_features.iloc[test_list_index_2[i]])

In [ ]:
################INICIO DE DEFINICIÓN DE PARÁMETROS DE LOS MODELOS CON GRID SERCH CV#####################
#se harán dos modelos, uno para cada variable target

############################# y1=H1N1 variable target################################
from sklearn.linear_model import RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from numpy import logspace
from sklearn.preprocessing import StandardScaler

# Definir tus datos x_train y y_train

lambdas = logspace(-11, -2, 10)
C_ = [0.01, 0.1, 1, 10]  # How many errors can admit SVM
gamma_ = [0.01, 0.1, 1, 10]  # How curved the decision boundaries will be
alpha_ = [0.01, 0.1, 1, 10]  # Naive Bayes

# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('tree', DecisionTreeClassifier(splitter="best")),
    ('svm', SVC()),
    ('nv', BernoulliNB())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "tree": {"tree__max_depth": [5, 7, 10], "tree__ccp_alpha": [0.1, 0.25, 0.5], "tree__max_features": [10, 15, 20]},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf", "linear"]},
    "nv": {"nv__alpha": alpha_}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y1)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

In [ ]:
#Selecting the best model
lead_board = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board =lead_board.sort_values(by="score", ascending=False)
#l_board.to_csv("lead_board_h1n1.csv")

In [ ]:
l_board

In [19]:
############################# y2=seasonal variable target################################


lambdas = logspace(-11, -2, 10)
C_ = [0.01, 0.1, 1, 10]  # How many errors can admit SVM
gamma_ = [0.01, 0.1, 1, 10]  # How curved the decision boundaries will be
alpha_ = [0.01, 0.1, 1, 10]  # Naive Bayes

# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('tree', DecisionTreeClassifier(splitter="best")),
    ('svm', SVC()),
    ('nv', BernoulliNB())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "tree": {"tree__max_depth": [5, 7, 10], "tree__ccp_alpha": [0.1, 0.25, 0.5], "tree__max_features": [10, 15, 20]},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf", "linear"]},
    "nv": {"nv__alpha": alpha_}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y2)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 32 candidates, totalling 96 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [22]:
#Selecting the best model
lead_board_2 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_2 =lead_board_2.sort_values(by="score", ascending=False)
#l_board_2.to_csv("lead_board_seasonal.csv")

In [24]:
l_board_2

,model,params,score
2,svm,"{'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel...",0.854143
0,ridge,{'ridge__alphas': 1e-08},0.847609
3,nv,{'nv__alpha': 0.01},0.818433
1,tree,"{'tree__ccp_alpha': 0.1, 'tree__max_depth': 5,...",0.500000


In [34]:
a=lead_board_2.loc[2]
a["params"]

{'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}

In [64]:
############################# y1=H1N1 variable target################################
###APLICANDO DE NUEVO GRID SEARCH CV, PERO AHORA SOLO PARA AFINAR (SOLO VARIANDO 
#LOs HIPERPARÁMETROS DE REGRESIÓN).
#Y VERIFICAR SI SE PUEDE APLICAR EL MISMO MODELO PARA AMBAS VARIABLES TARGETS

C_ = [0.1]  # How many errors can admit SVM
gamma_ = [0.001]  # How curved the decision boundaries will be
lambdas = [0.001, 1e-08]


estimators = [
    ('ridge', RidgeClassifierCV()),
    ('svm', SVC())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf"]}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y1)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [72]:
#Selecting the best model
lead_board_3 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_3 =lead_board_3.sort_values(by="score", ascending=False)
#l_board_3.to_csv("svm_board_h1n1.csv")

In [73]:
l_board_3

,model,params,score
0,ridge,{'ridge__alphas': 0.001},0.847579
1,svm,"{'svm__C': 0.1, 'svm__gamma': 0.001, 'svm__ker...",0.844966


In [71]:
############################# y2=seasonal variable target################################
###APLICANDO DE NUEVO GRID SEARCH CV, PERO AHORA SOLO PARA AFINAR (SOLO VARIANDO 
#LOs HIPERPARÁMETROS DE REGRESIÓN).
#Y VERIFICAR SI SE PUEDE APLICAR EL MISMO MODELO PARA AMBAS VARIABLES TARGETS


C_ = [0.1]  # How many errors can admit SVM
gamma_ = [0.001]  # How curved the decision boundaries will be
lambdas = [0.001, 1e-08]
# Explanation of alpha hyperparameter:
# Consider W* word not present in the training set
# P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
#  = Number of training points such that w* word present and Y=1 / Number of training points where Y=1
#  = 0/Number of training points where Y=1
# So to get rid of this problem we do Laplace smoothing. we add alpha to numerator and denominator field.
# = 0 + alpha / Number of training points where Y=1 + (Number of class labels in cl

estimators = [
    ('ridge', RidgeClassifierCV()),
    ('svm', SVC())
]

params = {
    "ridge": {"ridge__alphas": lambdas},
    "svm": {"svm__C": C_, "svm__gamma": gamma_, "svm__kernel": ["rbf"]}
}

best_params = {}
best_score = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for est_name, est in estimators:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (est_name, est)
    ])
    grid = GridSearchCV(pipeline, param_grid=params[est_name], scoring="roc_auc", cv=skf, verbose=4, n_jobs=-1)
    grid.fit(x_training_features, y2)
    best_params[est_name] = grid.best_params_
    best_score.append(grid.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [69]:
#Selecting the best model
lead_board_4 = pd.DataFrame({'model':list(best_params.keys()),
                           'params':list(best_params.values()),
                           'score':best_score})
l_board_4 =lead_board_4.sort_values(by="score", ascending=False)
#l_board_4.to_csv("svm_board_seasonal.csv")

In [70]:
l_board_4

,model,params,score
1,svm,"{'svm__C': 0.1, 'svm__gamma': 0.001, 'svm__ker...",0.821622
0,ridge,{'ridge__alphas': 0.001},0.820940


In [ ]:
#SE OBSERVA QUE PARA H1N1 VARIABLE TARGET ES MEJOR ridge	{'ridge__alphas': 0.001}
#SE OBSERVA QUE PARA SEASONAL VARIABLE TARGET ES MEJOR svm	{'svm__C': 0.1, 'svm__gamma': 0.001,}
#POR ENDE SE DETERMINA USAR DIFERENTES MODELOS

In [85]:
#######Determinando el mejor learning rate previo ha aplicar adaboost, usando h1n1 target#### 

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define los valores de learning_rate a probar
param_grid = {'clf__learning_rate': [0.1, 0.2]}

# Crea un estimador AdaBoostClassifier con LogisticRegression y SVC como clasificadores base
logistic_base = LogisticRegression(C=0.001, solver='lbfgs', max_iter=1000)
svm_base = SVC(C=0.1, gamma=0.001, probability=True)

clf = AdaBoostClassifier(base_estimator=logistic_base, n_estimators=100)

# Crea un pipeline con StandardScaler y el clasificador
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Realiza la búsqueda en rejilla con StratifiedKFold y roc_auc como métrica
grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kfold, scoring='roc_auc')
grid_search.fit(x_training_features, y1)

# El mejor valor de learning_rate
best_learning_rate = grid_search.best_params_['clf__learning_rate']

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-p

In [86]:
best_learning_rate
#es 0.2

0.2

In [90]:
###########################Aplicación de adaboost, usando h1n1 target###############
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import pandas as pd

# Define tu learning_rate
learning_rate = 0.2

# Crea un estimador AdaBoostClassifier con LogisticRegression y SVC como clasificadores base
logistic_base = LogisticRegression(C=0.001, solver='lbfgs', max_iter=1000)
svm_base = SVC(C=0.1, gamma=0.001, probability=True)

clf = AdaBoostClassifier(base_estimator=logistic_base, n_estimators=100, learning_rate=learning_rate)

# Crea un pipeline con StandardScaler y el clasificador
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Inicializa una lista para almacenar los resultados de ROC AUC
roc_auc_scores = []

# Itera sobre los diferentes pliegues (folds)
for train_index, test_index in stratified_kfold.split(x_training_features, y1):
    x_train_fold, x_test_fold = x_training_features.iloc[train_index], x_training_features.iloc[test_index]
    y_train_fold, y_test_fold = y1[train_index], y1[test_index]
    
    # Entrena el modelo
    pipeline.fit(x_train_fold, y_train_fold)
    
    # Calcula las probabilidades
    y_pred_proba = pipeline.predict_proba(x_test_fold)[:, 1]
    
    # Calcula y almacena el ROC AUC
    roc_auc = roc_auc_score(y_test_fold, y_pred_proba)
    roc_auc_scores.append(roc_auc)

# Imprime los resultados de ROC AUC
print(f'ROC AUC scores: {roc_auc_scores}')
print(f'Mean ROC AUC: {sum(roc_auc_scores) / len(roc_auc_scores)}')

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ROC AUC scores: [0.8188670235992246, 0.7870681232977553, 0.8001378403089836, 0.7914018362691435, 0.7767416382302605]
Mean ROC AUC: 0.7948432923410735


In [25]:

#Se intenta aplicar GradientBoostingClassifier:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import pandas as pd

# Crea un estimador GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(random_state=1234,learning_rate=0.1)

# Define los hiperparámetros a probar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Define el pipeline solo con StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', gb_clf)
])

# Define StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Realiza la búsqueda en rejilla con StratifiedKFold y roc_auc como métrica
grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kfold, scoring='roc_auc')
grid_search.fit(x_training_features.values, y1)

# Obtiene el mejor modelo
best_model = grid_search.best_estimator_

# Obtén las probabilidades (asegúrate de que x_test esté en el mismo formato que x_training_features)
x_test_scaled = grid_search.best_estimator_.named_steps['scaler'].transform(x_test)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

# Calcula el ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC AUC en test: {roc_auc}')

# Los mejores hiperparámetros
best_params = grid_search.best_params_
print(f'Los mejores hiperparámetros son: {best_params}')

ValueError: Invalid parameter 'learning_rate' for estimator Pipeline(steps=[('clf', GradientBoostingClassifier(random_state=1234))]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
#############################preparando el dataset a predecir##################################
df_test_original=pd.read_csv("test_set_features.csv")
col=pd.read_csv("training_set_features_eda_notnulls.csv").columns.tolist()
col=col[2:]
df_test_original=df_test_original[col]

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import pandas as pd

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=stratified_kfold, n_jobs=-1)

clf.fit(x_training_features.values, y1)
print(clf.score(x_training_features.values, y1))
print(clf.best_params_)

correct_test = correct_data(test)
testX = correct_test[predictor].values
result = clf.predict(testX)

test["Survived"] = result
result = test[["PassengerId", "Survived"]]

C:\Users\auror\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
211680 fits failed out of a total of 423360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
104698 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\auror\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 420, in fit
    self._validate_params()
  File "C:\Users\auror\anaconda3\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\auror\anaconda3\lib\site

0.814992324109784
{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 0.13636363636363638, 'n_estimators': 10, 'subsample': 1.0}


NameError: name 'correct_data' is not defined

In [ ]:
# Importar bibliotecas
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Definir los hiperparámetros
parametros = {
    'criterion': 'friedman_mse',
    'learning_rate': 0.2,
    'loss': 'deviance',
    'max_depth': 5,
    'max_features': 'log2',
    'min_samples_leaf': 0.1,
    'min_samples_split': 0.13636363636363638,
    'n_estimators': 10,
    'subsample': 1.0
}

# Crear el clasificador
clf = GradientBoostingClassifier(**parametros)

# Definir los datos X e y (supongo que ya los tienes cargados)
# X = ...
# y = ...

# Inicializar StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Listas para almacenar resultados
roc_auc_scores = []
predicted_probs = []

# Iterar sobre las divisiones
for train_index, test_index in skf.split(X, y):
    X_train, X_test = x_training_features_no_outliers.iloc[train_index], x_training_features_no_outliers.iloc[test_index]
    y_train, y_test = y1[train_index], y1[test_index]
    
    # Entrenar el modelo
    clf.fit(X_train, y_train)
    
    # Predecir probabilidades
    y_prob = clf.predict_proba(X_test)[:, 1]
    predicted_probs.extend(y_prob)
    
    # Calcular ROC AUC
    roc_auc = roc_auc_score(y_test, y_prob)
    roc_auc_scores.append(roc_auc)

# Calcular el promedio de ROC AUC
avg_roc_auc = np.mean(roc_auc_scores)

# Imprimir el resultado
print(f'ROC AUC promedio: {avg_roc_auc}')

# Imprimir las probabilidades continuas
print('Probabilidades Continuas:')
print(predicted_probs)

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Definir los hiperparámetros
parametros = {
    'criterion': 'friedman_mse',
    'learning_rate': 0.2,
    'loss': 'deviance',
    'max_depth': 5,
    'max_features': 'log2',
    'min_samples_leaf': 0.1,
    'min_samples_split': 0.13636363636363638,
    'n_estimators': 10,
    'subsample': 1.0
}

# Crear el clasificador
clf = GradientBoostingClassifier(**parametros)

# Definir los datos X e y
X = x_training_features_no_outliers
y = y_training_features_no_outliers["h1n1_vaccine"]

# Inicializar StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Listas para almacenar resultados
roc_auc_scores = []
predicted_probs = []

# Iterar sobre las divisiones
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Entrenar el modelo
    clf.fit(X_train, y_train)
    
    # Predecir probabilidades
    y_prob = clf.predict_proba(X_test)[:, 1]
    predicted_probs.extend(y_prob)
    
    # Calcular ROC AUC
    roc_auc = roc_auc_score(y_test, y_prob)
    roc_auc_scores.append(roc_auc)

# Calcular el promedio de ROC AUC
avg_roc_auc = np.mean(roc_auc_scores)

# Imprimir el resultado
print(f'ROC AUC promedio: {avg_roc_auc}')

# Imprimir las probabilidades continuas
print('Probabilidades Continuas:')
print(predicted_probs)

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is e

ROC AUC promedio: 0.7988071601158585
Probabilidades Continuas:
[0.10855914636968257, 0.22639488509944347, 0.2605869111340902, 0.11350486523980349, 0.11632327744997886, 0.07363345254293997, 0.3700224536115688, 0.21317466511290656, 0.07983482905230721, 0.38434342248812003, 0.0770341701559317, 0.12020597041002386, 0.2519568981800296, 0.3665605076213983, 0.24825113768956542, 0.40067486371352434, 0.3781128150019116, 0.0969625766969754, 0.3172949967115278, 0.11185730211535756, 0.0838133281559056, 0.16813875155645266, 0.19579048405972518, 0.10426964647007231, 0.28652120819710536, 0.1892633358184119, 0.33409518715447556, 0.3760933665566361, 0.20807084266447032, 0.20689758075092832, 0.437115062331493, 0.16928281200550657, 0.192639859891671, 0.1382086408667483, 0.19349441896171904, 0.09872873887501092, 0.22557558951594964, 0.11378586424139396, 0.23816334655724353, 0.20954824618899978, 0.17099707396081948, 0.35074693837748727, 0.07796570572593228, 0.18888951056058206, 0.19899936403092333, 0.24127

C:\Users\auror\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


In [ ]:
"""0.814992324109784
{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 
'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 0.13636363636363638, 
'n_estimators': 10, 'subsample': 1.0}"""

In [31]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Definir una función para construir el modelo
def build_model(optimizer='adam', activation='relu', hidden_units=32):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=x_training_features.shape[1], activation=activation))
    model.add(Dense(1, activation='sigmoid'))  # Capa de salida para clasificación binaria
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Crear un clasificador Keras para usar con GridSearchCV
model = KerasClassifier(build_fn=build_model, epochs=10, batch_size=32, verbose=0)

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
])

# Definir los parámetros para grid search
parametros = {
    'model__optimizer': ['adam', 'sgd'],
    'model__activation': ['relu', 'tanh'],
    'model__hidden_units': [16, 32, 64]
}

# Inicializar StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=skf, scoring='roc_auc')

# Realizar la búsqueda de parámetros
grid_result = grid_search.fit(x_training_features, y1)

# Obtener las predicciones y probabilidades continuas
y_pred = grid_result.predict(x_training_features)
y_prob = grid_result.predict_proba(x_training_features)[:, 1]  # Tomamos solo la columna correspondiente a la clase positiva

# Calcular el ROC AUC Score
roc_auc = roc_auc_score(y1, y_prob)

# Mostrar los resultados
print("Mejor Puntuación de AUC-ROC:", grid_result.best_score_)
print("Mejores Parámetros:", grid_result.best_params_)
print("ROC AUC Score:", roc_auc)

C:\Users\auror\AppData\Local\Temp\ipykernel_18088\3915030757.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model, epochs=10, batch_size=32, verbose=0)


835/835 [==============================] - 2s 2ms/step
Mejor Puntuación de AUC-ROC: 0.827020927892077
Mejores Parámetros: {'model__activation': 'relu', 'model__hidden_units': 16, 'model__optimizer': 'adam'}
ROC AUC Score: 0.8414209942611456


In [30]:
"""Mejor Puntuación de AUC-ROC: 0.827020927892077
Mejores Parámetros: {'model__activation': 'relu', 'model__hidden_units': 16, 'model__optimizer': 'adam'}
ROC AUC Score: 0.8414209942611456"""

array([[0.96333206, 0.03666794],
       [0.4924345 , 0.5075655 ],
       [0.97783417, 0.02216581],
       ...,
       [0.89312464, 0.10687538],
       [0.9833312 , 0.01666879],
       [0.81215596, 0.18784407]], dtype=float32)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Definir el clasificador base (Decision Tree)
base_classifier = DecisionTreeClassifier()

# Crear el clasificador Bagging
bagging_clf = BaggingClassifier(base_classifier, random_state=42)

# Definir el pipeline con la estandarización
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', bagging_clf)
])

# Definir los parámetros para grid search
parametros = {
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_samples': [0.5, 0.8, 1.0]
}

# Inicializar StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=skf, scoring='roc_auc', return_train_score=True)

# Definir los datos X e y
X = x_training_features
y = y1

# Realizar la búsqueda de parámetros
grid_result = grid_search.fit(X, y)

# Obtener las mejores configuraciones
best_estimator = grid_result.best_estimator_
best_params = grid_result.best_params_

# Calcular el ROC AUC Score
y_prob = best_estimator.predict_proba(X)[:, 1]
roc_auc = roc_auc_score(y, y_prob)

# Mostrar los resultados
print("Mejores Parámetros:", best_params)
print("ROC AUC Score:", roc_auc)

In [35]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Define los clasificadores que deseas probar como base
classifiers = [
    {
        'classifier': SVC(probability=True),
        'params': {'classifier__C': [1], 'classifier__gamma': [0.01], 'classifier__kernel': ['rbf']}
    },
    {
        'classifier': LogisticRegression(max_iter=1000),
        'params': {'classifier__C': [0.001], 'classifier__solver': ['lbfgs']}
    },
    {
        'classifier': DecisionTreeClassifier(),
        'params': {'classifier__max_depth': [None]}
    }
]

# Inicializa una lista para almacenar los resultados
results = []

# Itera sobre los clasificadores
for clf_info in classifiers:
    classifier = clf_info['classifier']
    parametros = clf_info['params']
    
    # Construye el pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', classifier)
    ])

    # Inicializa GridSearchCV
    grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=5, scoring='roc_auc')

    # Convierte el DataFrame a un array
    X_array = x_training_features.values

    # Realiza la búsqueda de parámetros
    grid_result = grid_search.fit(X_array, y1)

    # Obtiene las mejores configuraciones
    best_estimator = grid_result.best_estimator_
    best_params = grid_result.best_params_

    # Obtiene las probabilidades continuas
    y_prob = best_estimator.predict_proba(X_array)[:, 1]

    # Calcula el ROC AUC Score
    roc_auc = roc_auc_score(y1, y_prob)

    # Almacena los resultados
    results.append({
        'classifier': classifier.__class__.__name__,
        'best_params': best_params,
        'roc_auc_score': roc_auc,
        'y_prob': y_prob  # Añade las probabilidades continuas
    })

# Imprime los resultados
for result in results:
    print(f"Classifier: {result['classifier']}")
    print(f"Best Parameters: {result['best_params']}")
    print(f"ROC AUC Score: {result['roc_auc_score']}\n")

Classifier: SVC
Best Parameters: {'classifier__C': 1, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf'}
ROC AUC Score: 0.8351890748715352

Classifier: LogisticRegression
Best Parameters: {'classifier__C': 0.001, 'classifier__solver': 'lbfgs'}
ROC AUC Score: 0.8241473680992861

Classifier: DecisionTreeClassifier
Best Parameters: {'classifier__max_depth': None}
ROC AUC Score: 0.9999999622930018



In [36]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Define el clasificador que deseas probar como base
classifier = DecisionTreeClassifier()

# Inicializa una lista para almacenar los resultados
results = []

# Define los hiperparámetros que deseas probar
parametros = {
    'classifier__max_depth': [None, 5, 10, 20],
    'classifier__min_samples_split': [2, 5, 10, 20],
    'classifier__min_samples_leaf': [1, 2, 4, 8],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__criterion': ['gini', 'entropy']
}

# Construye el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', classifier)
])

# Inicializa GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=5, scoring='roc_auc')

# Convierte el DataFrame a un array
X_array = x_training_features.values

# Realiza la búsqueda de parámetros
grid_result = grid_search.fit(X_array, y1)

# Obtiene las mejores configuraciones
best_estimator = grid_result.best_estimator_
best_params = grid_result.best_params_

# Obtiene las probabilidades continuas
y_prob = best_estimator.predict_proba(X_array)[:, 1]

# Calcula el ROC AUC Score
roc_auc = roc_auc_score(y1, y_prob)

# Almacena los resultados
results.append({
    'classifier': classifier.__class__.__name__,
    'best_params': best_params,
    'roc_auc_score': roc_auc,
    'y_prob': y_prob  # Añade las probabilidades continuas
})

# Imprime los resultados
for result in results:
    print(f"Classifier: {result['classifier']}")
    print(f"Best Parameters: {result['best_params']}")
    print(f"ROC AUC Score: {result['roc_auc_score']}\n")

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

...

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


Classifier: DecisionTreeClassifier
Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 20}
ROC AUC Score: 0.830371289415607



C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


In [ ]:
"""Classifier: DecisionTreeClassifier
Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 20}
ROC AUC Score: 0.830371289415607"""

In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Define los clasificadores de base que deseas usar
classifiers = [
    ('svc', SVC(C=1, gamma=0.01, kernel='rbf', probability=True)),
    ('logistic_regression', LogisticRegression(max_iter=1000))
]

# Define el meta-clasificador
meta_classifier = DecisionTreeClassifier()

# Inicializa el StackingClassifier
stacking_classifier = StackingClassifier(estimators=classifiers, final_estimator=meta_classifier)

# Construye el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', stacking_classifier)
])

# Define los hiperparámetros que deseas probar
parametros = {
    'classifier__svc__C': [1],
    'classifier__svc__gamma': [0.01],
    'classifier__svc__kernel': ['rbf'],
    'classifier__logistic_regression__C': [0.001],
    'classifier__logistic_regression__solver': ['lbfgs']
}

# Inicializa GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=5, scoring='roc_auc')

# Convierte el DataFrame a un array
X_array = x_training_features.values

# Realiza la búsqueda de parámetros
grid_result = grid_search.fit(X_array, y1)

# Obtiene las mejores configuraciones
best_estimator = grid_result.best_estimator_
best_params = grid_result.best_params_

# Obtiene las probabilidades continuas
y_prob = best_estimator.predict_proba(X_array)[:, 1]

# Calcula el ROC AUC Score
roc_auc = roc_auc_score(y1, y_prob)

# Imprime los resultados
print(f"Best Parameters: {best_params}")
print(f"ROC AUC Score: {roc_auc}\n")

Best Parameters: {'classifier__logistic_regression__C': 0.001, 'classifier__logistic_regression__solver': 'lbfgs', 'classifier__svc__C': 1, 'classifier__svc__gamma': 0.01, 'classifier__svc__kernel': 'rbf'}
ROC AUC Score: 0.6536492807742021



In [45]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# Define los clasificadores de base que deseas usar
classifiers = [
    ('svc', SVC(C=1, gamma=0.01, kernel='rbf', probability=True)),
    ('DecisionTreeClassifier', DecisionTreeClassifier(criterion='gini', max_depth=10, max_features='auto', 
                                   min_samples_leaf=8, min_samples_split=20))
]

# Define el meta-clasificador y el estimador
meta_classifier = LogisticRegression(C=0.001, solver='lbfgs', max_iter=1000)

# Inicializa el StackingClassifier
stacking_classifier = StackingClassifier(estimators=classifiers, final_estimator=meta_classifier)

# Construye el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', stacking_classifier)
])

# Define los hiperparámetros que deseas probar
parametros = {
    'classifier__svc__C': [1],
    'classifier__svc__gamma': [0.01],
    'classifier__svc__kernel': ['rbf'],
    'classifier__DecisionTreeClassifier__criterion': ['gini'],
    'classifier__DecisionTreeClassifier__max_depth': [10],
    'classifier__DecisionTreeClassifier__max_features': ['auto'], 
    'classifier__DecisionTreeClassifier__min_samples_leaf': [8], 
    'classifier__DecisionTreeClassifier__min_samples_split': [20]
}

# Inicializa GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=5, scoring='roc_auc')

# Convierte el DataFrame a un array
X_array = x_training_features.values

# Realiza la búsqueda de parámetros
grid_result = grid_search.fit(X_array, y1)

# Obtiene las mejores configuraciones
best_estimator = grid_result.best_estimator_
best_params = grid_result.best_params_

# Obtiene las probabilidades continuas
y_prob = best_estimator.predict_proba(X_array)[:, 1]

# Calcula el ROC AUC Score
roc_auc = roc_auc_score(y1, y_prob)

# Imprime los resultados
print(f"Best Parameters: {best_params}")
print(f"ROC AUC Score: {roc_auc}\n")

C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\auror\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users

Best Parameters: {'classifier__DecisionTreeClassifier__criterion': 'gini', 'classifier__DecisionTreeClassifier__max_depth': 10, 'classifier__DecisionTreeClassifier__max_features': 'auto', 'classifier__DecisionTreeClassifier__min_samples_leaf': 8, 'classifier__DecisionTreeClassifier__min_samples_split': 20, 'classifier__svc__C': 1, 'classifier__svc__gamma': 0.01, 'classifier__svc__kernel': 'rbf'}
ROC AUC Score: 0.849216752746716



In [ ]:
"""Best Parameters: {'classifier__DecisionTreeClassifier__criterion': 'gini', 'classifier__DecisionTreeClassifier__max_depth': 10, 'classifier__DecisionTreeClassifier__max_features': 'auto', 'classifier__DecisionTreeClassifier__min_samples_leaf': 8, 'classifier__DecisionTreeClassifier__min_samples_split': 20, 'classifier__svc__C': 1, 'classifier__svc__gamma': 0.01, 'classifier__svc__kernel': 'rbf'}
ROC AUC Score: 0.849216752746716"""